In [1]:
import os
import pickle
from pathlib import Path
from typing import Final, Literal

from numpy import argmax, concatenate, unravel_index, vstack
from pandas import DataFrame, concat, read_csv, set_option, to_datetime
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

set_option("display.max_columns", None)

COLUMNS: Final[list[str]] = [
	"timestamp",
	"activity",
	"heart_rate",
	*[
		f"IMU_{body_part}_{suffix}"
		for body_part in ["hand", "chest", "ankle"]
		for suffix in [
			"temp_C",
			*[
				f"{scalar}_{axis}"
				for scalar in ["acc16g_ms^-2", "acc6g_ms^-2", "gyro_rad/s", "mag_μT"]
				for axis in ["x", "y", "z"]
			],
			*[f"orient_{x}" for x in range(1, 5)],
		]
	],
]
IMU_COLUMNS: Final[list[str]] = [
	col
	for col in COLUMNS
	if col.startswith("IMU_") and "acc6g_ms^-2" not in col and "orient" not in col
]

In [2]:
def read_w_log(path: Path, filename: str) -> tuple[DataFrame, str]:
	"""
	The IMU sensory data contains the following columns:
	- 1 temperature (°C)
	- 2...4 3D-acceleration data (ms^-2), scale: ±16g, resolution: 13-bit
	- 5...7 3D-acceleration data (ms^-2), scale: ±6g, resolution: 13-bit*
	- 8...10 3D-gyroscope data (rad/s)
	- 11...13 3D-magnetometer data (μT)
	- 14...17 orientation (invalid in this data collection)

	* This accelerometer is not precisely calibrated with the first one. Moreover, due
	to high impacts caused by certain movements (e.g. during running) with acceleration
	over 6g, it gets saturated sometimes. Therefore, the use of the data from the first
	accelerometer (with the scale of ±16g) is recommended.

	Args:
		path: Directory path containing the data file.
		filename: Name of the file to read.

	Returns:
		Tuple containing the cleaned DataFrame and subject ID (last 2 chars of filename).
	"""
	print(f"Reading: {filename}", end="\r")
	df = read_csv(os.path.join(path, filename), sep=r"\s+", header=None)
	df.columns = COLUMNS
	return (
		df.loc[
			:,
			~df.columns.str.contains("orient") & ~df.columns.str.contains("acc6g"),
		],
		filename.split(".")[0][-2:],
	)


def handle_nans(df: DataFrame) -> DataFrame:
	"""
	Handles NaN values in the sensor data with a time-series-aware strategy.

	- First, forward-fills to propagate the last valid observation.
	- Then, uses linear interpolation for short gaps.
	- Finally, drops any rows where sensor data is still missing.

	Args:
		df: The input DataFrame with potential NaN values.

	Returns:
		DataFrame with NaNs handled.
	"""
	df = df.copy()
	# For IMU data: linear interpolation for short gaps, drop for long gaps
	for col in IMU_COLUMNS:
		# Forward fill first (sensor readings typically persist briefly)
		df.loc[:, col] = df[col].ffill(limit=2)
		# Only interpolate if gap is ≤ 5 samples (0.05s at 100Hz)
		# IMU gaps can be interpolated without significant information loss.
		df.loc[:, col] = df[col].interpolate("linear", limit=5, limit_direction="both")
	# Drop rows where ANY IMU sensor still has NaN (likely sensor disconnection)
	return df.dropna(subset=IMU_COLUMNS)


def normalize_features(
	X_train: DataFrame, X_val: DataFrame, X_test: DataFrame, force_refit: bool = False
) -> tuple[DataFrame, DataFrame, DataFrame]:
	"""Normalizes IMU features using RobustScaler fitted on training data only.

	Args:
		X_train: Training feature DataFrame.
		X_val: Validation feature DataFrame.
		X_test: Test feature DataFrame.
		force_refit: If to ignores existing scaler and refits. Defaults to False.

	Returns:
		(normalized X_train, normalized X_val, normalized X_test, fitted scaler).

	Raises:
		FileNotFoundError: If scaler_path directory doesn't exist when trying to save.
		pickle.UnpicklingError: If saved scaler file is corrupted.
	"""
	scaler_path = Path("../data/PAMAP2/splits/robust_scaler.joblib")

	if scaler_path and scaler_path.exists() and not force_refit:
		with open(scaler_path, "rb") as f:
			scaler = pickle.load(f)
	else:
		scaler = RobustScaler().fit(X_train[IMU_COLUMNS])
		if scaler_path:
			scaler_path.parent.mkdir(parents=True, exist_ok=True)
			with open(scaler_path, "wb") as f:
				pickle.dump(scaler, f)

	X_train = X_train.copy()
	X_val = X_val.copy()
	X_test = X_test.copy()

	X_train.loc[:, IMU_COLUMNS] = scaler.transform(X_train[IMU_COLUMNS])
	X_val.loc[:, IMU_COLUMNS] = scaler.transform(X_val[IMU_COLUMNS])
	X_test.loc[:, IMU_COLUMNS] = scaler.transform(X_test[IMU_COLUMNS])

	return X_train, X_val, X_test

### _Mod proposal_: **Activity-Based Splitting**

> In novelty detection, you want to detect unseen patterns. If the same subject appears in both train and test, the model learns subject-specific characteristics, which won't generalize to new users.

In [3]:
def load_data(path: Path) -> tuple[DataFrame, DataFrame]:
	data, labels = [], []
	for df, subject in [  # all protocol files
		read_w_log(path, filename)
		for filename in os.listdir(path)
		if filename.endswith(".dat")
	]:  # droping rope jumping (24) cause only subject 9 does this activity
		df = handle_nans(df[~df["activity"].isin([0, 24])])
		df["subject"] = str(subject)
		df["timestamp"] = to_datetime(df["timestamp"], unit="s").dt.time
		df["id"] = df["subject"] + "_" + df["timestamp"].astype(str)

		data.append(df.drop(columns=["activity", "heart_rate"]))
		labels.append(df[["id", "activity"]])  # Index & Activity

	data, labels = concat(data), concat(labels)
	data["subject"] = data["subject"].astype("category")
	labels["activity"] = labels["activity"].astype("category")

	return data, labels

In [4]:
data, labels = load_data(Path("../data/PAMAP2_Dataset/Protocol/"))

data.to_csv("../data/PAMAP2/data.csv", index=False)
labels.to_csv("../data/PAMAP2/labels.csv", index=False)

df = data.merge(labels, how="left", on="id")
df.head()

,timestamp,IMU_hand_temp_C,IMU_hand_acc16g_ms^-2_x,IMU_hand_acc16g_ms^-2_y,IMU_hand_acc16g_ms^-2_z,IMU_hand_gyro_rad/s_x,IMU_hand_gyro_rad/s_y,IMU_hand_gyro_rad/s_z,IMU_hand_mag_μT_x,IMU_hand_mag_μT_y,IMU_hand_mag_μT_z,IMU_chest_temp_C,IMU_chest_acc16g_ms^-2_x,IMU_chest_acc16g_ms^-2_y,IMU_chest_acc16g_ms^-2_z,IMU_chest_gyro_rad/s_x,IMU_chest_gyro_rad/s_y,IMU_chest_gyro_rad/s_z,IMU_chest_mag_μT_x,IMU_chest_mag_μT_y,IMU_chest_mag_μT_z,IMU_ankle_temp_C,IMU_ankle_acc16g_ms^-2_x,IMU_ankle_acc16g_ms^-2_y,IMU_ankle_acc16g_ms^-2_z,IMU_ankle_gyro_rad/s_x,IMU_ankle_gyro_rad/s_y,IMU_ankle_gyro_rad/s_z,IMU_ankle_mag_μT_x,IMU_ankle_mag_μT_y,IMU_ankle_mag_μT_z,subject,id,activity
0,00:00:37.660000,30.375,2.21530,8.27915,5.58753,-0.004750,0.037579,-0.011145,8.93200,-67.9326,-19.9755,32.1875,0.124482,9.65003,-1.65181,0.036668,0.016559,-0.052791,0.567566,-50.7269,44.2728,30.75,9.73855,-1.84761,0.095156,0.002908,-0.027714,0.001752,-61.1081,-36.8636,-58.3696,01,01_00:00:37.660000,1
1,00:00:37.670000,30.375,2.29196,7.67288,5.74467,-0.171710,0.025479,-0.009538,9.58300,-67.9584,-20.9091,32.1875,0.200711,9.64980,-1.65043,0.019343,-0.024304,-0.059843,0.904990,-50.5080,43.5427,30.75,9.69762,-1.88438,-0.020804,0.020882,0.000945,0.006007,-60.8916,-36.3197,-58.3656,01,01_00:00:37.670000,1
2,00:00:37.680000,30.375,2.29090,7.14240,5.82342,-0.238241,0.011214,0.000831,9.05516,-67.4017,-19.5083,32.1875,0.270277,9.72331,-1.88174,-0.001428,0.038466,-0.046464,0.455480,-50.7209,44.0259,30.75,9.69633,-1.92203,-0.059173,-0.035392,-0.052422,-0.004882,-60.3407,-35.7842,-58.6119,01,01_00:00:37.680000,1
3,00:00:37.690000,30.375,2.21800,7.14365,5.89930,-0.192912,0.019053,0.013374,9.92698,-67.4387,-20.5602,32.1875,0.236737,9.72447,-1.72746,0.017277,-0.048547,-0.074946,0.324284,-50.1544,43.6570,30.75,9.66370,-1.84714,0.094385,-0.032514,-0.018844,0.026950,-60.7646,-37.1028,-57.8799,01,01_00:00:37.690000,1
4,00:00:37.700000,30.375,2.30106,7.25857,6.09259,-0.069961,-0.018328,0.004582,9.15626,-67.1825,-20.0857,32.1875,0.352225,9.72437,-1.68665,0.000275,-0.013352,-0.039315,0.462317,-50.7110,42.9228,30.75,9.77578,-1.88582,0.095775,0.001351,-0.048878,-0.006328,-60.2040,-37.1225,-57.8847,01,01_00:00:37.700000,1


In [5]:
def select_most_distinct_activities(
	data: DataFrame,
	labels: DataFrame,
	n_activities: int = 3,
	method: Literal["pca", "statistical", "variance"] = "pca",
) -> list[int]:
	"""
	Select the most distinct activities based on pairwise feature distances.

	Uses a greedy algorithm to maximize separation between selected activities:
	1. Start with the two activities that are furthest apart
	2. If selecting 3, add the activity with maximum minimum distance to the first two

	Args:
		data: Feature DataFrame
		labels: Labels DataFrame with 'id' and 'activity' columns
		n_activities: Number of activities to select (2 or 3)
		method: Method to use for activity representation:
			- 'pca': Use principal components (captures main movement patterns)
			- 'statistical': Use mean, std, and quartiles (robust statistics)
			- 'variance': Use variance and energy (good for dynamic activities)

	Returns:
		List of activity IDs representing the most distinct activities.

	Raises:
		ValueError: If method is not one of 'pca', 'statistical', or 'variance'.
	"""
	print(f"\n🔍 Selecting {n_activities} most distinct activities using '{method}'...")
	df = data.merge(labels, how="left", on="id")

	activity_stats = {}
	for activity in df["activity"].unique():
		activity_data = df[df["activity"] == activity][IMU_COLUMNS]
		if method == "pca":  # Reduce dimensionality and capture main characteristics
			activity_stats[activity] = (
				PCA(n_components=min(10, len(IMU_COLUMNS), len(activity_data)))
				.fit_transform(activity_data)  # PCA components as activity signature
				.mean(axis=0)
			)
		elif method == "statistical":
			activity_stats[activity] = concatenate(
				[  # Statistical features (more robust to outliers)
					activity_data.mean().values,
					activity_data.std().values,
					activity_data.quantile(0.25).values,
					activity_data.quantile(0.75).values,
				]
			)
		elif method == "variance":
			activity_stats[activity] = concatenate(
				[  # Variance and energy characteristics (good for dynamic activities)
					activity_data.var().values,
					activity_data.abs().mean().values,  # Mean absolute value (energy)
				]
			)
	activities = list(activity_stats.keys())
	distances = squareform(  # Calculate pairwise distances between activities
		pdist(vstack([activity_stats[act] for act in activities]), metric="euclidean")
	)
	selected_indices = []  # Greedy selection algorithm
	# Start with the pair that has maximum distance
	max_dist_idx = unravel_index(distances.argmax(), distances.shape)
	selected_indices.extend([max_dist_idx[0], max_dist_idx[1]])
	# If a third activity is needed, select the one with maximum minimum distance
	if n_activities == 3:
		remaining_indices = [
			i for i in range(len(activities)) if i not in selected_indices
		]
		# Select the activity with maximum minimum distance (most distinct from both)
		selected_indices.append(
			remaining_indices[
				argmax(
					[  # Find minimum distance to any already selected activity
						min(distances[idx, sel_idx] for sel_idx in selected_indices)
						for idx in remaining_indices
					]
				)
			]
		)
	print(f"\n✅ Selected {n_activities} most distinct activities:")
	for i, act in enumerate(
		selected_activities := [activities[i] for i in selected_indices], 1
	):
		print(f"  {i}. Activity {act}: {(df['activity'] == act).sum():,} samples")

	print("\n📏 Pairwise distances between selected activities:")
	for i in range(len(selected_indices)):
		for j in range(i + 1, len(selected_indices)):
			print(
				f"Activity {selected_activities[i]} ↔ Activity {selected_activities[j]}"
				f": {distances[selected_indices[i], selected_indices[j]]:.2f}"
			)
	return selected_activities

In [6]:
def activity_based_split(
	data: DataFrame,
	labels: DataFrame,
	test_activities: list[int] | None = None,
	n_distinct: int = 2,
	method: Literal["pca", "statistical", "variance"] = "pca",
	val_size: float = 0.2,
) -> tuple[DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
	"""
	Split data into train/val/test sets based on activities for novelty detection.

	Strategy:
	- Test set: Contains ONLY the most distinct activities (novel/unseen)
	- Train set: Contains all other activities (normal behavior)
	- Val set: Last 20% of train set in temporal order (time-series split)

	This ensures the model is evaluated on truly novel activities during testing.

	Args:
		data: Feature DataFrame
		labels: Labels DataFrame with 'id' and 'activity' columns
		test_activities: Optional list of activity IDs to use as test set.
				If None, automatically selects using distinctiveness algorithm.
		n_distinct: Number of activities to auto-select (if test_activities=None)
		method: How to calculate distinctiveness. 'pca', 'statistical' or 'variance'
		val_size: Proportion of train data to use for validation (e.g., 0.2 = 20%)

	Returns:
		tuple: (X_train, X_val, X_test, y_train, y_val, y_test)
	"""
	if test_activities is None:  # Auto-select distinct activities if not provided
		test_activities = select_most_distinct_activities(
			data, labels, n_activities=n_distinct, method=method
		)
	df = data.merge(labels, how="left", on="id")  # Merge data and labels
	test_mask = df["activity"].isin(test_activities)  # Split based on activities
	# Use temporal split for train/val: last 20% becomes validation
	train_val_df = df[~test_mask].copy()
	split_idx = int(len(train_val_df) * (1 - val_size))

	train_df = train_val_df.iloc[:split_idx].copy()
	val_df = train_val_df.iloc[split_idx:].copy()
	test_df = df[test_mask].copy()
	# Separate features and labels
	feature_cols = [col for col in data.columns if col != "id"]

	return (
		train_df[feature_cols].reset_index(drop=True),
		val_df[feature_cols].reset_index(drop=True),
		test_df[feature_cols].reset_index(drop=True),
		train_df[["id", "activity"]].reset_index(drop=True),
		val_df[["id", "activity"]].reset_index(drop=True),
		test_df[["id", "activity"]].reset_index(drop=True),
	)

In [7]:
X_train, X_val, X_test, y_train, y_val, y_test = activity_based_split(
	data=data, labels=labels, n_distinct=3, method="pca", val_size=0.2
)
# Normalize features using training data statistics only
X_train, X_val, X_test = normalize_features(X_train, X_val, X_test)

output_dir = Path("../data/PAMAP2/splits/")
output_dir.mkdir(parents=True, exist_ok=True)

X_train.to_csv(output_dir / "X_train.csv", index=False)
X_val.to_csv(output_dir / "X_val.csv", index=False)
X_test.to_csv(output_dir / "X_test.csv", index=False)

y_train.to_csv(output_dir / "y_train.csv", index=False)
y_val.to_csv(output_dir / "y_val.csv", index=False)
y_test.to_csv(output_dir / "y_test.csv", index=False)


🔍 Selecting 3 most distinct activities using 'pca'...

✅ Selected 3 most distinct activities:
  1. Activity 7: 188,107 samples
  2. Activity 6: 164,600 samples
  3. Activity 12: 117,216 samples

📏 Pairwise distances between selected activities:
Activity 7 ↔ Activity 6: 0.00
Activity 7 ↔ Activity 12: 0.00
Activity 6 ↔ Activity 12: 0.00
